# Central API Notebook

[![](https://files.readme.io/ca270e4-small-aruba-developer-hub_681x1002.png)](https://developer.arubanetworks.com/aruba-central/)

Find the documentation here: https://developer.arubanetworks.com/aruba-central/

Find the API reference here: https://developer.arubanetworks.com/aruba-central/reference/

## Import packages

In [ ]:
from decorest import GET, RestClient, backend, query, on, timeout

from CentralTokenAuth.CentralTokenAuth import CentralTokenAuth

## Add Central Client with decorest

Decorest is decorator heavy rest client. See [here](https://github.com/bkryza/decorest).

In [ ]:
# Central Client with API Definitions
@backend('httpx')
class Central(RestClient):
    @GET('monitoring/v2/clients')
    @query('group')
    @query('client_type')
    @query('client_status')
    @on(200, lambda r: r.json())
    @timeout(60)
    def get_clients(self, group, client_type, client_status='CONNECTED'):
        """
        This is a unified form of List APIs /monitoring/v1/clients/wired and /monitoring/v1/clients/wireless and it is backward compatible with these v1 apis.
        
        Get a list of unified clients. You can only specify one of group, swarm_id, cluster_id, network, site and label parameters.

        ---
        https://developer.arubanetworks.com/aruba-central/reference/apiexternal_controllerget_v2_unified_clients
        """

## Configure Endpoint/Client ID/Credential
### endpoint.json
``` json
{
    "name": "Internal",
    "base_url": "https://*-apigw.central.arubanetworks.com"
}
```
### client_id.json
``` json
{
    "client_id": "94a...",
    "client_secret": "bcn..."
}

```
### credential.json
``` json
{
    "access_token": "FqY...",
    "appname": "nms",
    "authenticated_userid": "user.name@example.com",
    "created_at": 1677142162,
    "credential_id": "085...",
    "expires_in": 7200,
    "id": "85f...",
    "refresh_token": "jx5...",
    "scope": "all",
    "token_type": "bearer"
}

```

In [ ]:
endpoint_file = './endpoint.json'
client_id_file = './client_id.json'
credential_file = './credential.json'

In [ ]:
import json
# Extract base_url from endpoint configuration file
base_url = None
with open(endpoint_file, 'r') as f:
    base_url = json.load(f)['base_url']

## Configure central api client
Configure central api client using credential provided via `client_id_file` and `credential_file`

In [ ]:
central_client = Central(base_url,
                         backend='httpx',
                         auth=CentralTokenAuth(
                             base_url=base_url,
                             client_id_file=client_id_file,
                             credential_file=credential_file))

## Load data via api

Using group `'Home-AP-Group'`, display `'WIRELESS'` or `'WIRED'` that are `'CONNECTED'` or `'FAILED_TO_CONNECT'`

In [ ]:
group = 'Home-AP-Group'

clients_response = central_client.get_clients(group, 'WIRELESS', 'CONNECTED')

clients_response

## Load clients into pandas DataFrame

Pandas is a module for easy manipulation of columnized data. See [here](https://pandas.pydata.org/about/index.html) and [here](https://pandas.pydata.org/docs/user_guide/index.html).

In [ ]:
import pandas as pd, csv

clients = pd.DataFrame(clients_response['clients'])

clients

## Export raw entries to csv

In [ ]:
clients.to_csv('clients_raw.csv')

## Transform timestamp to date string

Python uses seconds while central provides milliseconds. Therefore we divide by `1000.0`.

To change the formtat use `date_format`. For documentation see [here](https://docs.python.org/3.3/library/datetime.html#strftime-and-strptime-behavior).

In [ ]:
from datetime import datetime

date_format = '%Y-%m-%d %H:%M:%S CET'


def date_formatter(timestamp):
    dt = datetime.fromtimestamp(timestamp / 1000.0)
    date_text = dt.strftime(date_format)
    return date_text


clients['last_connection_datetime'] = clients['last_connection_time'].apply(
    date_formatter)

## Select columns, name them and export

`columns` is used to select which columns should be exported
`headers` provides the labels/headers for those columns

`columns` and `headers` must be in the same order.

We disable the `index` to remove the numbering from the output

In [ ]:
columns = [
    'associated_device_mac', 'associated_device_name', 'network',
    'last_connection_datetime', 'name'
]
headers = ['MAC Address', 'Device Name', 'SSID', 'Connect Time', 'Username']

clients.to_csv('output.csv', columns=columns, header=headers, index=False, doublequote=True, quoting=csv.QUOTE_NONNUMERIC)